In [ ]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="exaone3.5:2.4b")

AIMessage(content='한국의 수도는 **서울**입니다.', additional_kwargs={}, response_metadata={'model': 'exaone3.5:2.4b', 'created_at': '2025-04-05T16:50:59.721327Z', 'done': True, 'done_reason': 'stop', 'total_duration': 393810083, 'load_duration': 28303000, 'prompt_eval_count': 41, 'prompt_eval_duration': 175605292, 'eval_count': 10, 'eval_duration': 186671500, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a259f29b-17a9-49ef-9267-1abeb48c5d2b-0', usage_metadata={'input_tokens': 41, 'output_tokens': 10, 'total_tokens': 51})

In [ ]:
# RAG 사용 X. 단순 exaone3.5 모델을 사용한 예제
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

message_list = [
    SystemMessage(content="너는 공주대학교 님다시큐리티 동아리 규칙을 알려주는 어시스턴트야"),
    HumanMessage(content="님다시큐리티 동아리의 4번째 규칙을 알려줘"),
]

llm.invoke(message_list)

AIMessage(content='안녕하세요! 공주대학교 님다시큐리티 동아리에 대한 정보를 제공해 드리겠습니다. 하지만, 현재 제공 가능한 정보에는 구체적인 동아리 규칙 내용, 특히 4번째 규칙에 대한 상세 내용이 포함되어 있지 않습니다. 동아리 규칙은 시간이 지남에 따라 변경될 수도 있고, 정확한 내용을 알기 위해서는 동아리 내에서 직접 확인하거나, 동아리 활동에 참여하거나, 최근 활동 보고서나 공지사항을 참고하는 것이 가장 정확할 것입니다.\n\n만약 특정 규칙이나 최근 활동에 대한 정보가 필요하시다면, 다음과 같은 방법들을 추천드립니다:\n\n1. **동아리 웹사이트나 SNS 페이지 확인**: 많은 동아리들이 공식 웹사이트나 소셜 미디어 채널을 통해 최신 정보를 공유합니다.\n2. **동아리 선배나 멤버와의 직접 대화**: 동아리 내에서 선배 회원들과 이야기를 나누면 가장 정확한 정보를 얻을 수 있습니다.\n3. **동아리 활동 참여 또는 문의**: 동아리 활동에 참여하거나 직접 문의를 통해 정보를 얻는 것도 좋은 방법입니다.\n\n이런 방법들을 통해 필요한 정보를 얻으시길 바랍니다! 추가로 궁금한 점이 있으면 언제든지 말씀해 주세요.', additional_kwargs={}, response_metadata={'model': 'exaone3.5:2.4b', 'created_at': '2025-04-05T17:13:51.919122Z', 'done': True, 'done_reason': 'stop', 'total_duration': 7373112875, 'load_duration': 688731542, 'prompt_eval_count': 49, 'prompt_eval_duration': 1250852041, 'eval_count': 271, 'eval_duration': 5432016125, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-7

In [1]:
#from langchain.embeddings import OpenAIEmbeddings
#from langchain.chat_models import ChatOllama

from langchain.embeddings import HuggingFaceEmbeddings # Embedding Model
from langchain.vectorstores import Chroma # Vector Store
from langchain.chains import RetrievalQA # RetrievalQA Chain


In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install chromadb

In [20]:
# RAG 사용 O. 
# Logic1 문서 목록 : 동아리 규칙 
docs = [
    "1. 회비납부는 의무이며, 회비를 낸 인원만이 동아리 활동의 혜택 및 지원을 받을 수 있다.",
    "2. 회의 참석은 의무이며, 부득이한 경우 사전에 연락을 한다.",
    "3. 동아리실은 항상 청결하게 유지되어야 한다.",
    "4. 동아리 비품, 재산은 동아리원 공동의 재산이므로 밖으로 가지고 나가거나 개인용도로서의 사용을 엄금한다.",
    "5. 동아리원끼리의 다툼은 절대 엄금하며, 감정적인 다툼이 있을 시 마음에 담아두지 말고 그 날 안에 해결한다.",
    "6. 동아리 활동의 참여와 기여도가 높을지라도 불건전한 인품, 성품을 지닌 자는 동아리원 으로서의 자격을 박탈한다",
    "7. 동아리 내 활동이나 모임에서 술을 강요하거나 심한 군기를 잡는 행위는 금한다.",
    "8. 동아리 내에서 학우들에게 성에 대한 희롱, 추행 혹은 이와 유사한 언행을 삼가 할 것이다.",
]

# Logic2 임베딩 및 벡터 스토어 구축 
# vector = embedding.embed_query(docs)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Chroma를 사용한 벡터 스토어 구축 
vectorstore = Chroma.from_texts(docs, embedding, collection_name="club_rules")
# 벡터 스토어 기반 retriver 생성 (질의 시 가장 유사한 문서 1개 검색)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# Logic3 RetrievalQA 체인 구성 ?
# RetrievalQA 체인은 사용자 질의에 대해 retriever가 관련 문서를 검색하고, LLM이 이를 기반으로 답변을 생성합니다.
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)
# 검색된 문서를 그대로 LLM 프롬프트에 넣어 처리 

# Logic4 사용자 질의 실행 
query = "님다 시큐리티 동아리의 4번째 규칙을 알려줘"
result = qa_chain.run(query)
print("Ans : ", result)




/var/folders/_z/lk5nb_w55c7clyq38bbzkzz40000gn/T/ipykernel_80282/1736682631.py:34: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(query)


Ans :  님다 시큐리티 동아리의 4번째 규칙은 동아리의 비품과 재산이 동아리원들 사이에서 공동으로 소유되고 관리되어야 하며, 이를 개인 용도로 사용하거나 동아리 외부로 가져가는 것을 엄격히 금지하고 있습니다. 즉, 동아리 활동과 관련된 장비나 자산은 모두 동아리 활동에 활용되어야 하며, 개인적으로 이용하거나 동아리를 벗어나 사용하는 것은 허용되지 않습니다.
